In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re
import json
import time
import random

import spotipy
import tweepy

from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from collections import defaultdict
from collections import deque
from fake_useragent import UserAgent

%matplotlib inline

In [2]:
test = pd.read_csv("helper/test.csv")
train = pd.read_csv("helper/train.csv")
artists = pd.read_csv("helper/artists.csv")
profiles = pd.read_csv("helper/profiles.csv")

In [85]:
print "All profiles: " + str(len(profiles))
median_age = profiles.age.median()
profiles['age'] = [median_age if pd.isnull(age) else age for age in profiles.age]
profiles = profiles[(profiles.age <= 60) & (profiles.age >= 12)]
print "New profiles: " + str(len(profiles))

All profiles: 233286
New profiles: 231368


In [238]:
merged = pd.merge(train, profiles, on='user')

somelist = []
for group in merged.groupby('user') :
    somelist.append(group[1]['plays'].median())
GLOBAL_USER = np.median(somelist)

somelist = []
for group in merged.groupby('artist') :
    somelist.append(group[1]['plays'].median())
GLOBAL_ARTIST = np.median(somelist)

somelist = []
for group in merged.groupby('country') :
    somelist.append(group[1]['plays'].median())
GLOBAL_COUNTRY = np.median(somelist)

somelist = []
for group in merged.groupby('sex') :
    somelist.append(group[1]['plays'].median())
GLOBAL_SEX = np.median(somelist)

In [4]:
train_sample = train.sample(frac=0.01)
#train_sample.plays = np.log(train_sample.plays)
test_sample = train.sample(frac=0.005)
#test_sample.plays = np.log(test_sample.plays)

In [6]:
def produce_df(train_sample, test_sample, profiles, GLOBAL_MEDIAN, returning="train") :
    train_merged = pd.merge(train_sample, profiles, on="user")
    test_merged = pd.merge(test_sample, profiles, on="user")
    
    grouped_medians = defaultdict(lambda:GLOBAL_MEDIAN)
    for user in train_merged.groupby('user') :
        grouped_medians[user[0]] = user[1]['plays'].median()
    for artist in train_merged.groupby('artist') :
        grouped_medians[artist[0]] = artist[1]['plays'].median()
    for age in train_merged.groupby('age') :
        grouped_medians[age[0]] = age[1]['plays'].median()
    for country in train_merged.groupby('country') :
        grouped_medians[country[0]] = country[1]['plays'].median()
    for sex in train_merged.groupby('sex') :
        grouped_medians[sex[0]] = sex[1]['plays'].median()
        
    test_merged['user_median'] = [grouped_medians[user] for user in test_merged.user]
    test_merged['artist_median'] = [grouped_medians[artist] for artist in test_merged.artist]
    test_merged['age_median'] = [grouped_medians[age] for age in test_merged.age]
    test_merged['country_median'] = [grouped_medians[country] for country in test_merged.country]
    test_merged['sex_median'] = [grouped_medians[sex] for sex in test_merged.sex]

    train_merged['user_median'] = [grouped_medians[user] for user in train_merged.user]
    train_merged['artist_median'] = [grouped_medians[artist] for artist in train_merged.artist]
    train_merged['age_median'] = [grouped_medians[age] for age in train_merged.age]
    train_merged['country_median'] = [grouped_medians[country] for country in train_merged.country]
    train_merged['sex_median'] = [grouped_medians[sex] for sex in train_merged.sex]
    
    if returning == "train" :
        return train_merged[['user_median','artist_median','age_median','country_median','sex_median']], train_merged['plays']
    elif returning == "test" :
        return test_merged[['user_median','artist_median','age_median','country_median','sex_median']], test_merged['plays']
    else :
        return "Choose train or test"

In [7]:
#GLOBAL_MEDIAN = np.log(train.plays.median())
GLOBAL_MEDIAN = 116
X_train, y_train = produce_df(train_sample, test_sample, profiles, GLOBAL_MEDIAN, returning="train")
X_test, y_test = produce_df(train_sample, test_sample, profiles, GLOBAL_MEDIAN, returning="test")
print X_train.shape
print y_train.shape
print X_test.shape

(41548, 5)
(41548,)
(20774, 5)


In [ ]:
%time
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

CPU times: user 8 µs, sys: 10 µs, total: 18 µs
Wall time: 14.1 µs


In [ ]:
lm = LogisticRegression()
lm.fit(X_train, y_train)
y_lm_pred = lm.predict(X_test)

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_gnb_pred = gnb.predict(X_test)

In [ ]:
svc = LinearSVC(C=1.0)
svc.fit(X_train, y_train)
y_svc_pred = svc.predict(X_test)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
y_rfc_pred = rfc.predict(X_test)

In [ ]:
#MAE(np.exp(y_pred), np.exp(y_test))
MAE(y_rfc_pred, y_test)

In [503]:
a_merged = train.merge(profiles)
b_merged = test.merge(profiles)

In [517]:
a_merged[a_merged.plays > 10000].sort('plays',ascending=False)

,user,artist,plays,sex,age,country
3013946,8d0384537845e7f2b1b8b3e8a9f67eb8d9439794,dcaa4f81-bfb7-44eb-8594-4e74f004b6e4,419157,m,31,Austria
947353,baee6143fb184f2014f05c737c28cd57d6d0486f,79239441-bfd5-4981-a70c-55c3f15c1287,242328,m,31,Brazil
294489,2a088c171293674c39d0e02f495c6f91362357b0,35f866dc-c061-48ba-8157-cf2e0eac4857,232650,NaN,29,New Zealand
2981914,d5c1fe51720f69baae1281e09efbc9080d7c304f,00a9f935-ba93-4fc8-a33a-993abe9c936b,179843,f,25,Russian Federation
610979,8519545b80b95da502233743fa9bd1ff941ab13f,197450cd-0124-4164-b723-3c22dd16494d,155482,m,46,United States
2923731,c46c9683e1ac940b53b8ea52ac27ce160fade9a2,fbb375f9-48bb-4635-824e-4120273b3ba7,144827,f,18,Canada
3432866,d1f816fb11d62d24dd9a3babe4bb2e49b5444bbc,5b11f4ce-a62d-471e-81fc-a69a8278c7da,136695,m,44,Brazil
1997356,09d12dfa05a0852053a9017121034a837fa4019e,4d7928cd-7ed2-4282-8c29-c0c9f966f1bd,134993,m,39,United Kingdom
3039231,54027330bb09fe82d2b7e85749d4041eab3d5e4e,e69c8ccb-25b1-4c4a-89c6-8fdad1172951,129630,f,23,United States
1972513,d0e63bd9ecdde014b0a389fe8ffeecc305428ad2,cc197bad-dc9c-440d-a5b5-d52ba2e14234,118857,m,17,Italy


In [512]:
y_pred[y_pred > 10000]

array([ 16815.80636285,  18221.45830469,  18196.17055375,  19106.69654295,
        19144.46396318,  13017.42543201,  12995.42180457,  23371.16147913,
        13287.55130793,  13318.42206881,  10364.33619764,  12052.32752427,
        11882.23708965,  14965.96518696,  18766.17242635,  18786.53399204,
        10481.58103946,  10362.31099894,  11416.27801629,  15906.96441448,
        16602.86888677,  16619.94632896,  18852.7536753 ,  12735.53139624,
        13084.7273011 ,  10578.8576063 ,  29304.54886983,  12670.579136  ,
        10233.20666892,  10500.20576894])

In [510]:
len(y_test[y_test > 10000])

92